In [7]:
rm(list=ls())

# apt 가격 데이터 셋
### setwd 
# setwd("C:/Users/kdg95/Documents")

### packages
library(caret)
library(e1071)
library(dplyr)
library(mice)

install.packages("lime")

##################################################################################3
################과제###############################################################
#####################################################################################
### 여러가지 모델을 형성 하고 stacking 적용하는 함수 만들기
# 데이터 불러오기 (수정하지 말것)
# df <- read.csv('apt.csv')  # 학습용 데이터
# test <- read.csv('mid_X.csv')  # 테스트용 데이터(실제 테스트는 중간테스트 데이터와 파일명은 같고 내용은 다름)
apt_train <- read.csv('data/train_apt.csv')
apt_test <- read.csv('data/test_apt.csv')

apt_test_numeric = apt_test[sapply(apt_test, is.numeric)]
apt_test_factor = apt_test[sapply(apt_test, is.factor)]

apt_train_numeric = apt_train[sapply(apt_train, is.numeric)]
apt_train_factor = apt_train[sapply(apt_train, is.factor)]

# apt_train_factor
# sapply(apt_train_numeric, is.numeric)

levels(apt_train_factor$asile_type) <- c(NA, "A", "M", "S", "T")
levels(apt_train_factor$earthquake) <- c(NA, "N", "Y")
levels(apt_train_factor$heat_source) <- c(NA, "CHP", "LNG", "LPG", "OIL")
levels(apt_train_factor$heat_type) <- c(NA, "C", "I", "R")
mc <- mice(apt_train_numeric[,!names(apt_train_numeric) %in% 'price'], method='cart',1)
mc2 <- mice(apt_train_factor[,!names(apt_train_factor) %in% 'price'], method='cart',1)

# mice_list                   

mice_output1 <- complete(mc)
mice_output2 <- complete(mc2)
heat_source_dm = dummyVars('~ heat_source', mice_output2)
heat_source_dm = data.frame(predict(heat_source_dm, mice_output2))
heat_type_dm = dummyVars('~ heat_type', mice_output2)
heat_type_dm = data.frame(predict(heat_type_dm, mice_output2))
earthquake_dm = dummyVars('~ earthquake', mice_output2)
earthquake_dm = data.frame(predict(earthquake_dm, mice_output2))
asile_dm = dummyVars('~ asile_type', mice_output2)
asile_dm = data.frame(predict(asile_dm, mice_output2))
apt_real_train <- cbind(heat_source_dm,heat_type_dm,earthquake_dm, asile_dm, mice_output1)
apt_real_train$price = apt_train$price

#### 아파트 년식을 라벨링 한 다음 더미화시켰다.
bins = c(1980,1985,1990,1995,2000,2005,2010,2015)
labels = c("1","2","3","4","5","6","7")
drops <- c("permission_date")

permission_date_train <- as.character(apt_real_train$permission_date)
permission_date_train <- data.frame(substr(permission_date_train, 1, 4))
names(permission_date_train)[1] <- paste("permission_date") 
permission_date_train$permission_date <- strtoi(permission_date_train$permission_date)
permission_date_train$label <- cut(permission_date_train$permission_date, bins, labels = labels)
permission_date_dm = dummyVars('~ label', permission_date_train)
permission_date_dm = data.frame(predict(permission_date_dm, permission_date_train))
apt_real_train <- cbind(apt_real_train[,!(names(apt_real_train) %in% drops)], permission_date_dm)

##### test 전처리
levels(apt_test_factor$asile_type) <- c(NA, "A", "M", "S", "T")
levels(apt_test_factor$earthquake) <- c(NA, "N", "Y")
levels(apt_test_factor$heat_source) <- c(NA, "CHP", "LNG", "LPG", "OIL")
levels(apt_test_factor$heat_type) <- c(NA, "C", "I", "R")
apt_test_factor$heat_source[is.na(apt_test_factor$heat_source)] <- "OIL"
mc3 <- mice(apt_test_factor[,!names(apt_test_factor) %in% 'price'], method='cart',1)
mc4 <- mice(apt_test_numeric[,!names(apt_test_numeric) %in% 'price'], method='cart',1)

mice_output3 <- complete(mc3)
mice_output4 <- complete(mc4)
heat_source_dm1 = dummyVars('~ heat_source', mice_output3)
heat_source_dm1 = data.frame(predict(heat_source_dm1, mice_output3))
heat_type_dm1 = dummyVars('~ heat_type', mice_output3)
heat_type_dm1 = data.frame(predict(heat_type_dm1, mice_output3))
earthquake_dm1 = dummyVars('~ earthquake', mice_output3)
earthquake_dm1 = data.frame(predict(earthquake_dm1, mice_output3))
asile_dm1 = dummyVars('~ asile_type', mice_output3)
asile_dm1 = data.frame(predict(asile_dm1, mice_output3))
apt_real_test <- cbind(heat_source_dm1,heat_type_dm1,earthquake_dm1, asile_dm1, mice_output4)

bins = c(1980,1985,1990,1995,2000,2005,2010,2015)
labels = c("1","2","3","4","5","6","7")
drops <- c("permission_date")

permission_date_test <- as.character(apt_real_test$permission_date)
permission_date_test <- data.frame(substr(permission_date_test, 1, 4))
names(permission_date_test)[1] <- paste("permission_date") 
permission_date_test$permission_date <- strtoi(permission_date_test$permission_date)
permission_date_test$label <- cut(permission_date_test$permission_date, bins, labels = labels)
permission_date_dm1 = dummyVars('~ label', permission_date_test)
permission_date_dm1 = data.frame(predict(permission_date_dm1, permission_date_test))
apt_real_test <- cbind(apt_real_test[,!(names(apt_real_test) %in% drops)], permission_date_dm1)

apt_real_test

write.csv(apt_real_train, "miced_apt_train.csv")
write.csv(apt_real_test, "miced_apt_test.csv")

miced_apt_train <- read.csv("miced_apt_train.csv")
miced_apt_test <- read.csv("miced_apt_test.csv")

train_data = miced_apt_train[-c(1)]
train_data = train_data[complete.cases(train_data),]
test_data = miced_apt_test[-c(1)]
test_data = test_data[complete.cases(test_data),]

idx <- createDataPartition(y = train_data$price, p = 0.7, list =FALSE)
X_train <- train_data[idx,]
X_test <- train_data[-idx,]

xgb.grid = expand.grid(
  nrounds = c(300,500),
  eta = c(0.03,0.05),
  gamma = c(3,5),
  max_depth = c(4,6),
  min_child_weight = c(6,8),
  colsample_bytree = c(0.3,0.5),
  subsample = c(0.2,0.6)
)


control = trainControl(method = 'cv', search = 'random', number=2, verbose = TRUE)
gbm.model = train(price ~., data = X_train, tuneLength = 10, trControl = control, method="gbm")

control2 = trainControl(method='cv', search='grid', number=5,verbose = TRUE)
xgb.model <- train(
  price ~ .,
  data = X_train,
  tuneGrid = xgb.grid,
  trControl = control2,
  method = 'xgbTree'
)

rf.grid = expand.grid(
  .mtry = c(1,3,5)
)
control3 = trainControl(method='cv', search='grid', number=5,verbose = TRUE)
rf.model <- train(
  price ~ .,
  data = X_train,
  tuneGrid = rf.grid,
  trControl = control3,
  method = 'rf'
)

saveRDS(gbm.model, "gbmmodel.rds")

saveRDS(xgb.model, "xgbmodel.rds")
saveRDS(rf.model, "rfmodel.rds")

gbm_model <- readRDS('gbmmodel.rds')
xgb_model <- readRDS('xgbmodel.rds')
rf_model <- readRDS('rfmodel.rds')

y_test_data <- test_data

X_test_y <- X_test$price
drops <- c("price")
X_test_x2 <- X_test[ , !(names(X_test) %in% drops)]

calculate_adj.r2 <- function(df,b){
  n <- dim(df)[1]
  a <- as.numeric(df$price)
  b <- as.numeric(b)
  p <- dim(df)[2]
  denumerate <- sum((a-mean(a))**2)
  numerate <- sum((a-b)**2)
  r2 <- 1 - (numerate / denumerate)
  adj.r2 <- 1 - ((n-1)/(n-p))*(1-r2)


  return (adj.r2)
}

pred_xgb <- predict(xgb_model, X_test_x2)
pred_gbm <- predict(gbm_model, X_test_x2)
pred_rf <- predict(rf_model,X_test_x2)

predDF <- data.frame(pred_xgb, pred_gbm, pred_rf, price = X_test_y)

stack_control = trainControl(method = 'cv', search = 'random', number=2, verbose = TRUE)
stack_gbm.model = train(price ~., data = predDF, tuneLength = 10, trControl = stack_control, method="gbm")

calculate_adj.r2(predDF, predDF$pred_xgb)
calculate_adj.r2(predDF, predDF$pred_gbm)
calculate_adj.r2(predDF, predDF$pred_rf)

predict_test <- predict(stack_gbm.model, test_data)

write.csv(predict_test, "output_apt_test.csv")

Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

